In [1]:
import qsharp
import qsharp.azure
import matplotlib.pyplot as plt

targets = qsharp.azure.connect(
    resourceId="/subscriptions/31b21bff-585f-4485-aabe-5b2faad8abd8/resourceGroups/AzureQuantum/providers/Microsoft.Quantum/Workspaces/CSYE6305Assignments",
    location="eastus")

Preparing Q# environment...
.

Connecting to Azure Quantum...

Authenticated using Microsoft.Azure.Quantum.Authentication.TokenFileCredential


Connected to Azure Quantum workspace CSYE6305Assignments in location eastus.


In [2]:
%%qsharp

open Microsoft.Quantum.Math;
open Microsoft.Quantum.Canon;
open Microsoft.Quantum.Arrays;
open Microsoft.Quantum.Convert;
open Microsoft.Quantum.Logical;
open Microsoft.Quantum.Intrinsic;
open Microsoft.Quantum.Arithmetic;
open Microsoft.Quantum.Preparation;
open Microsoft.Quantum.Diagnostics;
open Microsoft.Quantum.Measurement;

In [3]:
%%qsharp

// Type to represen sum equality contriant
newtype SEC = (variables: Int[], sum: Int);

In [4]:
%%qsharp

// Helper function to check if a number is present in an array
function IsNumberPresentInArray(n : Int, array : Int[]) : Bool {
    return Any(EqualI(_, n), array);
}

// Helper function to find the common elements in two arrays
function FindCommonElementsInArrays(array1: Int[], array2: Int[]): Int[] {
    mutable result = [];
    for i in 0 .. Length(array1) - 1 {
        if IsNumberPresentInArray(array1[i], array2) {
            set result += [array1[i]];
        }
    }
    return result;
}

// Helper function to remove duplicates from an array
function RemoveDuplicates(arr: Int[]): Int[] {
    mutable result = [];
    for i in 0 .. Length(arr) - 1 {
        if not IsNumberPresentInArray(arr[i], result) {
            set result += [arr[i]];
        }
    }
    return result;
}

// Helper function to get list of possible options not in limitations
function GetLimitationConstraints(options: Int[], limitations: Int[]): Int[] {
    mutable result = [];
    for limitation in limitations {
        if not IsNumberPresentInArray(limitation, options) {
            set result += [limitation];
        }
    }
    return result;
}

In [5]:
%%qsharp

// Gets all the possible sum options for a given constraint
function SumOptions(constriant: SEC): Int[][] {
    mutable result = [];
    let variables = constriant::variables;
    let sumValue = constriant::sum;
    if Length(variables) == 2 {
        for i in 0 .. 3 {
            if sumValue - i < 4 and sumValue - i > -1 {
                if i != sumValue - i {
                    set result += [[i, sumValue - i]];
                }
            }
        }
    } elif Length(variables) == 3 {
        for i in 0 .. 3 {
            if sumValue - i < 4 and sumValue - i > -1 {
                let options = SumOptions(SEC([0,1], i));
                for o in options {
                    if sumValue - i != o[0] and sumValue - i != o[1] and o[0] != o[1] {
                        set result += [[sumValue-i, o[0], o[1]]];
                        set result += [[o[0], sumValue-i, o[1]]];
                        set result += [[o[0], o[1], sumValue-i]];
                    }
                }
            }
        }
    }
    return result;
}

In [6]:
%%qsharp

// Find the puzzle empty variables and there sums
// Then gets the sum constraints for each empty variable
function FindSumConstraints(sumEqualityConstraints: SEC[], nVariables: Int): (Int,Int)[] {
    mutable result = [];
    let limitations = [0, 1, 2, 3];
    for i in 0 .. nVariables - 1 {
        mutable x = 0;
        mutable options1 = [];
        mutable options2 = [];
        mutable finalOptions = [];
        for constriaint in sumEqualityConstraints {
            let variables = constriaint::variables;
            for j in 0 .. Length(variables) - 1 {
                if variables[j] == i {
                    if x == 0 {
                        for option in SumOptions(constriaint) {
                            set options1 += [option[j]];
                        }
                    } elif x == 1 {
                        for option in SumOptions(constriaint) {
                            set options2 += [option[j]];
                        }
                        set finalOptions = RemoveDuplicates(FindCommonElementsInArrays(options1, options2));
                    }
                    set x += 1;
                }
            }
        }
        Message($"Possible value(s) for X{i}: {finalOptions}");
        let LimitedOptions = GetLimitationConstraints(finalOptions, limitations);

        for option in LimitedOptions {
            set result += [(i, option)];
        }
    }

    Message($"Sum constraints: {result}");
    return result;
}

In [7]:
%%qsharp

// Read value from a register
operation MeasureValue (register: Qubit[]): Int {
    return MeasureInteger(LittleEndian(register));
}

// Read all values from a register
operation MeasureAllValues (valueQubits: Int, register: Qubit[]): Int[] {
    let nVariables = Length(register) / valueQubits;
    let valuePartitions = Partitioned([valueQubits, size=(nVariables - 1)], register);
    return ForEach(MeasureValue, valuePartitions);
}

In [8]:
%%qsharp

// Checks if the value of two variables are the same
operation ApplyValueEqualityOracle(vqb0: Qubit[], vqb1: Qubit[], target: Qubit): Unit is Adj + Ctl {
    within {
        // Iterates over pair of (vqb0, vqb1)
        // Then computes XOR of bits vqb0[i] and vqb1[i] in place (storing it in vqb1[i])
        ApplyToEachCA(CNOT, Zipped(vqb0, vqb1));
    } apply {
        // If all computed XORs are 0, then the values are equal, flip the target qubit
        ControlledOnInt(0, X)(vqb1, target);
    }
}

In [9]:
%%qsharp

// Checks if the variable values satisfy the inequality constraints
operation ApplyVariableValuesOracles (nVariables :Int, valueQubits: Int,
    inequalityConstraints: (Int, Int)[],
    valueRegister: Qubit[], target: Qubit
): Unit is Adj + Ctl {
    let nInequalityConstraints = Length(inequalityConstraints);
    use inequalityConflictQubits = Qubit[nInequalityConstraints];
    within {
        for ((start, end), conflictQubit) in Zipped(inequalityConstraints, inequalityConflictQubits) {
            // if the values are the same the result will be 1, indicating a conflict
            ApplyValueEqualityOracle(
                valueRegister[start * valueQubits .. (start + 1) * valueQubits - 1],
                valueRegister[end * valueQubits .. (end + 1) * valueQubits - 1],
                conflictQubit
            );
        }
    } apply {
        // If no conflicts (all qubits are in 0 state), the variable is valid
        ControlledOnInt(0, X)(inequalityConflictQubits, target);
    }
}

In [10]:
%%qsharp

// Convert the marking oracle to a phase oracle
operation ApplyPhaseOracle (oracle : ((Qubit[], Qubit) => Unit is Adj), register : Qubit[]): Unit is Adj {
    use target = Qubit();
    within {
        // Put the target into the |-⟩ state.
        X(target);
        H(target);
    } apply {
        // Apply the marking oracle; since the target is in the |-⟩ state,
        // flipping the target if the register satisfies the oracle condition
        // will apply a -1 factor to the state.
        oracle(register, target);
    // We put the target back into |0⟩ so we can return it.
    }
}

In [11]:
%%qsharp

operation ApplyGroversIteration(register: Qubit[], oracle: ((Qubit[], Qubit) => Unit is Adj), statePrep: (Qubit[] => Unit is Adj), iterations: Int): Unit {
    let applyPhaseOracle = ApplyPhaseOracle(oracle, _);
    statePrep(register);

    for _ in 1 .. iterations {
        applyPhaseOracle(register);
        within {
            Adjoint statePrep(register);
            ApplyToEachA(X, register);
        } apply {
            Controlled Z(Most(register), Tail(register));
        }
    }
}

// Grover search to find variable valid values
operation FindValuesWithGrover(nVariables: Int, valueQubits: Int, nIterations: Int,
    oracle:((Qubit[], Qubit) => Unit is Adj),
    statePrep: (Qubit[] => Unit is Adj)
): Int[] {
    // The value register has bits per value for each variable
    use register = Qubit[valueQubits * nVariables];
    Message($"Trying search with {nIterations} iterations...");
    if (nIterations > 75) {
        Message($"Warning: This might take a while");
    }
    ApplyGroversIteration(register, oracle, statePrep, nIterations);
    return MeasureAllValues(valueQubits, register);
}

In [12]:
%%qsharp

// Show the size of the search space, i.e. the number of possible combinations
function SearchSpaceSize(nVariables: Int, valueQubits: Int, sumConstraints: (Int, Int)[]): Int {
    mutable valueOptions = [1 <<< valueQubits, size=nVariables];
    for (cell, _) in sumConstraints {
        set valueOptions w/= cell <- valueOptions[cell] - 1;
    }
    return Fold(TimesI, 1, valueOptions);
}

// Estimate the number of iterations required for solution
// Kakuro puzzles only have one solution, so we can use the formula
function NIterations(searchSpaceSize: Int): Int {
    let angle = ArcSin(1. / Sqrt(IntAsDouble(searchSpaceSize)));
    let nIterations = Round(0.25 * PI() / angle - 0.5);
    return nIterations;
}

In [13]:
%%qsharp

// Encodes sum constraints into amplitudes
function AllowedAmplitudes(nVariables: Int, valueQubits: Int, sumConstraints: (Int, Int)[]) : Double[][] {
    mutable amplitudes = [[1.0, size=1 <<< valueQubits], size=nVariables];
    for (cell, value) in sumConstraints {
        set amplitudes w/= cell <- (amplitudes[cell] w/ value <- 0.0);
    }
    return amplitudes;
}

// Prepare an equal superposition of all basis states that satisfy the constraints
operation PrepareSearchStatesSuperposition(nVariables: Int, valueQubits: Int,
    sumConstraints: (Int, Int)[], register : Qubit[]
): Unit is Adj + Ctl {
    // Split the given register into nVariables chunks of size bits per value
    let valueRegister = Chunks(valueQubits, register);
    // For each variable, create an array of possible states we're looking at
    let amplitudes = AllowedAmplitudes(nVariables, valueQubits, sumConstraints);
    // For each variable, prepare a superposition of its possible states on the chunk storing its value
    for (amps, chunk) in Zipped(amplitudes, valueRegister) {
        PrepareArbitraryStateD(amps, LittleEndian(chunk));
    }
}

In [14]:
%%qsharp

function IsKakuroSolutionValid(inequalityConstraints: (Int,Int)[], sumEqualityConstraints: SEC[], values: Int[]) : Bool {
    if (Any(GreaterThanOrEqualI(_, 4), values)) {
        return false;
    }
    if (Any(EqualI, inequalityConstraints)) { 
        return false; 
    }

    for constraint in sumEqualityConstraints {
        let variables = constraint::variables;
        let expectedSum = constraint::sum;
        mutable valuesSum = 0;
        for variable in variables {
            set valuesSum += values[variable];
        }
        if valuesSum != expectedSum {
            return false;
        }
    }

    return true;
}

In [15]:
%%qsharp

// Checks if the value found for each empty variable is corrrect and satisfies all constraints
operation SolvePuzzle(nVariables: Int, valueQubits: Int, size: (Int,Int), inequalityConstraints: (Int,Int)[], sumEqualityConstraints: SEC[]): (Bool, Int[]) {
    let oracle = ApplyVariableValuesOracles(nVariables, valueQubits, inequalityConstraints, _, _);
    let sumConstraints = FindSumConstraints(sumEqualityConstraints, nVariables);
    let statePrep = PrepareSearchStatesSuperposition(nVariables, valueQubits, sumConstraints, _);
    let searchSpaceSize = SearchSpaceSize(nVariables, valueQubits, sumConstraints);
    let numIterations = NIterations(searchSpaceSize);
    let (nRows, nCols) = size;

    Message($"Running Quantum test with number of variables = {nVariables}");
    Message($"   Bits per value = {valueQubits}");
    Message($"   Inequality constraints = {inequalityConstraints}");
    Message($"   Sum Equality Constraints = {sumEqualityConstraints}");
    Message($"   Estimated number iterations needed = {numIterations}");
    Message($"   Size of kakuro grid = {nRows}x{nCols}");
    Message($"   Search space size = {searchSpaceSize}");
    let values = FindValuesWithGrover(nVariables, valueQubits, numIterations, oracle, statePrep);

    Message($"Got Sudoku solution: {values}");
    if (IsKakuroSolutionValid(inequalityConstraints, sumEqualityConstraints, values)) {
       Message($"Got valid Sudoku solution: {values}");
       return (true, values);
    } else {
       Message($"Got invalid Sudoku solution: {values}");
       return (false, values);
    }
}

In [16]:
%%qsharp

operation Start(): Unit {

    Message("Solving Kakuro Puzzle using Grover's Algorithm");

    // the size of the puzzle
    // let size = (4, 4);
    let size = (3, 3);

    // number of missing boxes
    // let nVariables = 7;
    let nVariables = 4;

    // bits to represent missing value for 0 .. 3
    let valueQubits = 2;

    // array of varibles that can not be equal
    // let inequalityConstraints = [(0,1), (0,2), (1,3), (1,5), (2,3), (2,4), (3,4), (3,5), (4,6), (5,6)];
    let inequalityConstraints = [(0,1), (0,2), (1,3), (2,3)];

    // array of sum equality constraints
    // let sumEqualityConstraints = [
    //     SEC([0,1], 3),
    //     SEC([0,2], 5),
    //     SEC([1,3,5], 3),
    //     SEC([2,3,4], 5),
    //     SEC([4,6], 1),
    //     SEC([5,6], 1)
    // ];
    let sumEqualityConstraints = [
        SEC([0,1], 4),
        SEC([0,2], 5),
        SEC([1,3], 1),
        SEC([2,3], 2)
    ];

    let result = SolvePuzzle(nVariables, valueQubits, size, inequalityConstraints, sumEqualityConstraints);
    let (isValid, values) = result;
    if isValid {
        for i in 0 .. nVariables - 1 {
            Message($"X{i} = {values[i]}");
        }
    }
}

In [17]:
%%timeit
Start.simulate()

Solving Kakuro Puzzle using Grover's Algorithm
Possible value(s) for X0: [3]
Possible value(s) for X1: [1]
Possible value(s) for X2: [2]
Possible value(s) for X3: [0]
Sum constraints: [(0, 0),(0, 1),(0, 2),(1, 0),(1, 2),(1, 3),(2, 0),(2, 1),(2, 3),(3, 1),(3, 2),(3, 3)]
Running Quantum test with number of variables = 4
   Bits per value = 2
   Inequality constraints = [(0, 1),(0, 2),(1, 3),(2, 3)]
   Sum Equality Constraints = [SEC(([0,1], 4)),SEC(([0,2], 5)),SEC(([1,3], 1)),SEC(([2,3], 2))]
   Estimated number iterations needed = 0
   Size of kakuro grid = 3x3
   Search space size = 1
Trying search with 0 iterations...
Got Sudoku solution: [3,1,2,0]
Got valid Sudoku solution: [3,1,2,0]
X0 = 3
X1 = 1
X2 = 2
X3 = 0
Solving Kakuro Puzzle using Grover's Algorithm
Possible value(s) for X0: [3]
Possible value(s) for X1: [1]
Possible value(s) for X2: [2]
Possible value(s) for X3: [0]
Sum constraints: [(0, 0),(0, 1),(0, 2),(1, 0),(1, 2),(1, 3),(2, 0),(2, 1),(2, 3),(3, 1),(3, 2),(3, 3)]
Running

In [18]:
%%timeit
Start.simulate_sparse()

Solving Kakuro Puzzle using Grover's Algorithm
Possible value(s) for X0: [3]
Possible value(s) for X1: [1]
Possible value(s) for X2: [2]
Possible value(s) for X3: [0]
Sum constraints: [(0, 0),(0, 1),(0, 2),(1, 0),(1, 2),(1, 3),(2, 0),(2, 1),(2, 3),(3, 1),(3, 2),(3, 3)]
Running Quantum test with number of variables = 4
   Bits per value = 2
   Inequality constraints = [(0, 1),(0, 2),(1, 3),(2, 3)]
   Sum Equality Constraints = [SEC(([0,1], 4)),SEC(([0,2], 5)),SEC(([1,3], 1)),SEC(([2,3], 2))]
   Estimated number iterations needed = 0
   Size of kakuro grid = 3x3
   Search space size = 1
Trying search with 0 iterations...
Got Sudoku solution: [3,1,2,0]
Got valid Sudoku solution: [3,1,2,0]
X0 = 3
X1 = 1
X2 = 2
X3 = 0
Solving Kakuro Puzzle using Grover's Algorithm
Possible value(s) for X0: [3]
Possible value(s) for X1: [1]
Possible value(s) for X2: [2]
Possible value(s) for X3: [0]
Sum constraints: [(0, 0),(0, 1),(0, 2),(1, 0),(1, 2),(1, 3),(2, 0),(2, 1),(2, 3),(3, 1),(3, 2),(3, 3)]
Running

In [19]:
# qsharp.azure.target("microsoft.estimator")
# qsharp.azure.submit(Start, jobName="Resource estmation for Sudoku problem solver small problem")

In [20]:
qsharp.azure.status("5e98e1f8-3649-4e56-a45a-3430d170480e")
resource_estimaion_result = qsharp.azure.output("5e98e1f8-3649-4e56-a45a-3430d170480e")
resource_estimaion_result

Physical qubits,240600,"Number of physical qubits This value represents the total number of physical qubits, which is the sum of 37800 physical qubits to implement the algorithm logic, and 202800 physical qubits to execute the T factories that are responsible to produce the T states that are consumed by the algorithm."
Runtime,18ms 168us,"Total runtime This is a runtime estimate (in nanosecond precision) for the execution time of the algorithm. In general, the execution time corresponds to the duration of one logical cycle (6us) multiplied by the 3028 logical cycles to run the algorithm. If however the duration of a single T factory (here: 67us 600ns) is larger than the algorithm runtime, we extend the number of logical cycles artificially in order to exceed the runtime of a single T factory."
Logical algorithmic qubits,84,"Number of logical qubits for the algorithm after layout Laying out the logical qubits in the presence of nearest-neighbor constraints requires additional logical qubits. In particular, to layout the $Q_{\rm alg} = 33$ logical qubits in the input algorithm, we require in total $2 \cdot Q_{\rm alg} + \lceil \sqrt{8 \cdot Q_{\rm alg}}\rceil + 1 = 84$ logical qubits."
Algorithmic depth,3028,"Number of logical cycles for the algorithm To execute the algorithm using Parallel Synthesis Sequential Pauli Computation (PSSPC), operations are scheduled in terms of multi-qubit Pauli measurements, for which assume an execution time of one logical cycle. Based on the input algorithm, we require one multi-qubit measurement for the 147 single-qubit measurements, the 165 arbitrary single-qubit rotations, and the 15 T gates, three multi-qubit measurements for each of the 154 CCZ and 133 CCiX gates in the input program, as well as 16 multi-qubit measurements for each of the 115 non-Clifford layers in which there is at least one single-qubit rotation with an arbitrary angle rotation."
Logical depth,3028,"Number of logical cycles performed This number is usually equal to the logical depth of the algorithm, which is 3028. However, in the case in which a single T factory is slower than the execution time of the algorithm, we adjust the logical cycle depth to exceed the T factory's execution time."
Number of T states,3803,"Number of T states consumed by the algorithm To execute the algorithm, we require one T state for each of the 15 T gates, four T states for each of the 154 CCZ and 133 CCiX gates, as well as 16 for each of the 165 single-qubit rotation gates with arbitrary angle rotation."
Number of T factories,15,Number of T factories capable of producing the demanded 3803 T states during the algorithm's runtime The total number of T factories 15 that are executed in parallel is computed as $\left\lceil\dfrac{3803\;\text{T states} \cdot 67us 600ns\;\text{T factory duration}}{1\;\text{T states per T factory} \cdot 18ms 168us\;\text{algorithm runtime}}\right\rceil$
Number of T factory invocations,254,"Number of times all T factories are invoked In order to prepare the 3803 T states, the 15 copies of the T factory are repeatedly invoked 254 times."
Physical algorithmic qubits,37800,Number of physical qubits for the algorithm after layout The 37800 are the product of the 84 logical qubits after layout and the 450 physical qubits that encode a single logical qubit.
Physical T factory qubits,202800,"Number of physical qubits for the T factories Each T factory requires 13520 physical qubits and we run 15 in parallel, therefore we need $202800 = 13520 \cdot 15$ qubits."
Required logical qubit error rate,1.31e-9,The minimum logical qubit error rate required to run the algorithm within the error budget The minimum logical qubit error rate is obtained by dividing the logical error probability 3.33e-4 by the product of 84 logical qubits and the total cycle count 3028.


In [25]:
qsharp.azure.target("ionq.qpu.aria-1")
qsharp.azure.submit(Start, jobName="Sudoku solver on ionq simulator")


Loading package Microsoft.Quantum.Providers.IonQ and dependencies...
Active target is now ionq.qpu.aria-1
Submitting Start to target ionq.qpu.aria-1...
fail: Microsoft.Quantum.IQSharp.AzureClient.AzureClient[0]
      Failed to submit Q# operation Start for execution.
      System.AggregateException: One or more errors occurred. (Exception has been thrown by the target of an invocation.)
       ---> System.Reflection.TargetInvocationException: Exception has been thrown by the target of an invocation.
       ---> Microsoft.Quantum.Providers.Core.Processor.CannotCompareMeasurementResultException: Measurement results cannot be compared to Zero or One on the target architecture
         at Microsoft.Quantum.Providers.IonQ.Processor.ResultPlaceholder.GetValue()
         at Microsoft.Quantum.Simulation.Core.Result.Equals(Result p) in D:\a\1\s\submodules\qsharp-runtime\src\Simulation\Core\Types.cs:line 102
         at Microsoft.Quantum.Convert.ResultAsBool.<>c.<get___Body__>b__6_0(Result __in_

Failed to submit Q# operation Start for execution.
Exception has been thrown by the target of an invocation.


AzureError: {'error_code': 1011, 'error_name': 'JobSubmissionFailed', 'error_description': 'Failed to submit the job to the Azure Quantum workspace.'}